In [1]:
import os
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
def connect_google_sheets(SPREADSHEET_ID):
    # connect to Google Sheets
    scopes = ['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive']
    secret_file = os.path.join(os.getcwd(), 'client_secret.json')
    credentials = Credentials.from_service_account_file(secret_file, scopes=scopes)
    gc = gspread.authorize(credentials)
    # open a google sheet
    # SPREADSHEET_ID = '1Hm2wU94SnTy5NYHljcolM8R17Pmoy2MzYGfy4igB6nI'
    gs = gc.open_by_key(SPREADSHEET_ID)
    return gs

In [5]:
import cx_Oracle
import pandas as pd
# my $DB_CONNECT_STRING = "DBI:Oracle:host=il01dbpn3;sid=VALORORA";
# my $DB_USR = "valordfmprd";
# my $DB_PWD = "oracl3";

conStr = 'valordfmprd/oracl3@il01dbpn3:1521/VALORORA'
conn = cx_Oracle.connect(conStr)
cur = conn.cursor()

query = """SELECT * FROM INVENTORY_BIN_DATA"""

cur.execute(query)
bin_audit_list = cur.fetchall()
bin_audit_df = pd.DataFrame(bin_audit_list)
print(bin_audit_df)
# do something like fetch, insert etc.
cur.close()
conn.close()



                        0     1         2        3       4     5          6  \
0                    None  None  10089507       Ok  1125.0  None     abc123   
1     2015-12-17 00:00:00  None  10089507       Ok     1.0  None       test   
2     0022-11-30 00:00:00  None  10089507       Ok     1.0  None       test   
3     0022-11-30 00:00:00  None  10089507       Ok  2222.0  None     abc123   
4     2022-11-30 00:00:00  None  10089507  MISSING     2.0  None        223   
...                   ...   ...       ...      ...     ...   ...        ...   
4428  2022-12-09 00:00:00  None    120144       Ok     0.0  None  920-0337B   
4429  2022-12-09 00:00:00  None    FWH784       Ok     0.0  None   wgp02489   
4430  2022-12-09 00:00:00  None    120144       Ok     0.0  None   wgp01472   
4431  2022-12-09 00:00:00  None    120144       Ok     0.0  None   WGP02775   
4432  2022-12-09 00:00:00  None    FWH784       Ok     0.0  None  100-0290C   

          7  
0      TEST  
1      TEST  
2      TE

In [2]:
import pandas as pd
import locale

def fixColToInt(doc_goods):
  locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
  for index, row in doc_goods.iterrows():
    if isinstance(row["Qty in UoE + sign (ERFME)"], int) == False:
      num_str = row["Qty in UoE + sign (ERFME)"].strip()
      if num_str.endswith("-") == True:
        num = locale.atoi(num_str.split("-")[0])
        doc_goods.at[index, "Qty in UoE + sign (ERFME)"] = -int(num)
  doc_goods.astype({'Qty in UoE + sign (ERFME)': 'int64'}) # convert column to integer
  return doc_goods

gs = connect_google_sheets("1L4q2Y_-4xakHRohoqqbhH9DkYNnjrjNGE4Qj1GCC9iI")

# get list of observe WO
observe_wo_ws = gs.worksheet('Observe WO')
work_orders = pd.DataFrame(observe_wo_ws.get_all_records())
work_orders.head()

# get exploded BOM for Observe
exploded_BOM_ws = gs.worksheet('Exploded BOM for Observe')
exploded_BOM = pd.DataFrame(exploded_BOM_ws.get_all_records())

# get WO Doc Goods Movements
doc_goods_ws = gs.worksheet('WO Doc Goods Movements')
doc_goods = pd.DataFrame(doc_goods_ws.get_all_records())
doc_goods = fixColToInt(doc_goods)

# Create Input column
exploded_BOM['Input'] = exploded_BOM.apply(lambda r: doc_goods.loc[doc_goods['Material'] == r['Component'], 'Qty in UoE + sign (ERFME)'].sum(), axis=1)
# Create Consumption column
exploded_BOM['Consumption'] = exploded_BOM['Input'] / exploded_BOM['Quantity']

In [3]:
exploded_BOM

,Material,Stack operation,Item,Component,Plant,Object description,Object description .1,Unit,Quantity,Item cat.,Input,Consumption
0,100-0209B,1,1,120-0382,2040,PCBA H4/P7 Dome IR Board,IC LED CC Boost Driver 1.2A Switch 6SON,EA,2,L,6400,3200.000000
1,100-0209B,1,2,130-0123,2040,PCBA H4/P7 Dome IR Board,RES 10K 1% 1/16W 0402,EA,2,L,683720,341860.000000
2,100-0209B,1,3,130-0135,2040,PCBA H4/P7 Dome IR Board,RES 0R 1% 1/16W 0402,EA,6,L,179536,29922.666667
3,100-0209B,1,4,130-0337,2040,PCBA H4/P7 Dome IR Board,RES 2.87R 1% 1/10W 0603,EA,2,L,4800,2400.000000
4,100-0209B,1,5,140-0142,2040,PCBA H4/P7 Dome IR Board,CAP 0.22uF 16V X7R 0402,EA,2,L,43112,21556.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
16652,WGA00346-200,1,12,WGP747-028,2040,"PCBA, WGA00346-200 4RE GPS VERSION 2","Cer Capacitor, 0.1 uF, 0402, +-10%, 16 V",EA,3,L,188823,62941.000000
16653,WGA00346-200,1,13,WGP747-062,2040,"PCBA, WGA00346-200 4RE GPS VERSION 2","Cer Capacitor, 10 uF, 0402, 20%, 6.3 V",EA,3,L,58635,19545.000000
16654,WGA00346-200,1,14,WGP748-001,2040,"PCBA, WGA00346-200 4RE GPS VERSION 2","Resistor, 0 Ohm, 0402, 0.063 W",EA,2,L,88922,44461.000000
16655,WGA00346-200,1,15,WGP748-004,2040,"PCBA, WGA00346-200 4RE GPS VERSION 2","Resistor, 1K Ohm, 1%, 0402, 0.063 W",EA,1,L,161869,161869.000000


In [4]:
from collections import defaultdict
orders_dict = defaultdict(list)
for index, row in work_orders.iterrows():
    wo_num = row['Order']
    current_order = work_orders.query('Order == {0}'.format(wo_num))
    order_output = current_order['Delivered quantity (GMEIN)']
    print(order_output)
    current_exp_BOM = exploded_BOM.loc[exploded_BOM['Material'].isin(current_order['Material'])].copy()

    # Create Output and Difference columns 
    current_exp_BOM['Output'] = int(order_output)
    current_exp_BOM['Difference'] = current_exp_BOM['Consumption'] - current_exp_BOM['Output']
    is_no_difference = (current_exp_BOM['Difference'] == 0).all() # check to see if the Difference column contains all 0, which means that this work order has no discrepancy
    if is_no_difference == False: # if there are discrepancies
        orders_dict[wo_num].append(current_order)
        orders_dict[wo_num].append(current_exp_BOM)

0    4000
Name: Delivered quantity (GMEIN), dtype: int64
1    3000
Name: Delivered quantity (GMEIN), dtype: int64
2    2000
Name: Delivered quantity (GMEIN), dtype: int64
3    2000
Name: Delivered quantity (GMEIN), dtype: int64
4    4000
Name: Delivered quantity (GMEIN), dtype: int64
5    12000
Name: Delivered quantity (GMEIN), dtype: int64
6    4000
Name: Delivered quantity (GMEIN), dtype: int64
7    3000
Name: Delivered quantity (GMEIN), dtype: int64
8    3712
Name: Delivered quantity (GMEIN), dtype: int64
9    5000
Name: Delivered quantity (GMEIN), dtype: int64
10    2000
Name: Delivered quantity (GMEIN), dtype: int64
11    3200
Name: Delivered quantity (GMEIN), dtype: int64
12    2400
Name: Delivered quantity (GMEIN), dtype: int64
13    6
Name: Delivered quantity (GMEIN), dtype: int64
14    12
Name: Delivered quantity (GMEIN), dtype: int64
15    1
Name: Delivered quantity (GMEIN), dtype: int64
16    30
Name: Delivered quantity (GMEIN), dtype: int64
17    200
Name: Delivered quantit

In [8]:
orders_dict[1186706][0] # shows the work order header
orders_dict[1186706][1] # shows the work order exploded bom and discrepancies calc

,Material,Stack operation,Item,Component,Plant,Object description,Object description .1,Unit,Quantity,Item cat.,Input,Consumption,Output,Difference
13328,VSM-48901,1,1,VSD-48901,2040,L5Q SOLAR-POWERED FIXED LPR,L5Q Solar-Powered Fixed LPR,EA,1,L,15,15.0,7,8.0
13329,VSM-48901,1,2,VSD-51422,2040,L5Q SOLAR-POWERED FIXED LPR,32GB SD Cards,EA,1,L,15,15.0,7,8.0
13330,VSM-48901,1,3,VSD-51431,2040,L5Q SOLAR-POWERED FIXED LPR,"5mm Hex Key, Short",EA,1,L,15,15.0,7,8.0
13331,VSM-48901,1,4,VSD-51449,2040,L5Q SOLAR-POWERED FIXED LPR,"Quik Release Clamp, Range 1-3/4"" to 16""",EA,4,L,60,15.0,7,8.0


In [14]:
# loop through all keys in dictionary
set(orders_dict.keys())

{1186706,
 1191497,
 1219106,
 1220789,
 1221274,
 1221497,
 1222300,
 '1222522',
 1222522,
 1222639,
 1224170,
 1224281,
 1226006,
 1226010,
 1226169,
 1226937,
 1226975,
 1226976,
 1227064,
 1227082,
 1227088,
 1227089,
 1227090,
 1227300,
 1227553,
 1227707,
 1227765,
 1227940,
 1227946,
 1227965,
 1227976,
 1227977,
 1227985,
 1228180,
 1228236,
 1228239,
 1228246,
 1228285,
 1228305,
 1228310,
 1228311,
 1228351,
 1228375,
 1228606,
 1228610,
 1228612,
 1228613,
 1228614,
 1228668,
 1228669,
 1228675,
 1228762,
 1228768,
 1228770,
 1228777,
 1228810,
 1228815,
 1228873,
 1228889,
 1228893,
 1228894,
 1228938,
 1228950,
 1228951,
 1228955,
 1228956,
 1228964,
 1228965,
 1228976,
 1229018,
 1229027,
 1229030,
 1229047,
 1229048,
 1229050,
 1229051,
 1229056,
 1229057,
 1229096,
 1229097,
 1229101,
 1229102,
 1229103,
 1229106,
 1229109,
 1229110,
 1229176,
 1229207,
 1229209,
 1229210,
 1229211,
 1229212,
 1229222,
 1229227,
 1229240,
 1229241,
 1229242,
 1229243,
 1229244,
 1229249